Nougat:
1. discard figure and put caption to the end of page
2. keep table as latex code, put to the end of page
3. table first, then figure captions

In [9]:
from transformers import NougatProcessor, VisionEncoderDecoderModel
from PIL import Image
import pymupdf
import torch

device = "cuda:7"
dtype = torch.bfloat16

model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base", device_map=device, torch_dtype=dtype)
processor = NougatProcessor.from_pretrained("facebook/nougat-base")

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    896,
    672
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "qkv_bias": true,
  "transformers_version": "4.47.0.dev0",
  "use_absolute_embeddings": false,
  "window_size": 7
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_final_layer_norm":

In [12]:
# sample image
pixmap = pymupdf.open("./2307.00421.pdf")[5].get_pixmap(dpi=800)
mode = 'RGB'
img = Image.frombytes(mode, [pixmap.width, pixmap.height], pixmap.samples)
nougat_inputs = processor(img, return_tensors="pt")
outputs_nougat = model.generate(
    nougat_inputs['pixel_values'].to(model.device, dtype=dtype),
    min_length=1,
    max_new_tokens=1024,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
)
generated_text = processor.batch_decode(outputs_nougat, skip_special_tokens=True)
print(generated_text[0])

### Effectiveness of different features in the physical world

#### 4.3.1 Color transfer and texture blurring



In physical-world deployments, attack patches are often affected by color transfer due to lighting conditions or blurring caused by camera focus or smudging. To analyze if these changes will affect the performance of the attack patches, we compare the performance of an original patch, a color-adjusted patch, and a local texture adjusted patch. The color-adjusted patch is applied by adding a value (\(\delta\)) to all values in RGB channels and making sure \(\delta\) will not lead to an overflow. This color transfer will not change the texture information of the patch. The local texture adjustment is applied by using a 3x3 Gaussian blur. Figure 4 demonstrates the two types of feature adjustments applying to a patch with brightness range=0.24.



The performances of different patches are shown in Table 1. Regardless of the lightness restriction, the color transfer patch achieve